# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [ ]:
import math
import os
import numpy as np
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
import sqlalchemy as db
from sqlalchemy import create_engine
import re
import datetime
import sqlite3

In [ ]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = ""
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Calculate distance

In [ ]:
def calculate_distance_with_coords(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude):
    lat1=pickup_longitude
    lon1=pickup_latitude
    lat2=dropoff_longitude
    lon2=dropoff_latitude
    
    R = 6371  # Earth's radius in kilometers

    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # calculate differences between latitudes and longitudes
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # calculate Haversine formula
    a = math.sin(d_lat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

Add Distance Column

In [ ]:
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(lambda row: calculate_distance_with_coords(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    return dataframe

### Process Taxi Data

In [ ]:
# Get the website page of the taxi_url
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [ ]:
# In the website page, find links for 'Yellow Taxi Trip Records'
# From 2009 to 2015 monthly data
def find_taxi_parquet_links():
    get_taxi_html()
    soup = bs4.BeautifulSoup(get_taxi_html(), 'html.parser')
    l1 = soup.find_all("a")
    l2 = []
    l3 = []
    pattern = []
    for i in range(len(l1)):
        if l1[i].text == 'Yellow Taxi Trip Records':
            l2.append(l1[i]['href'])
    for j in range(len(l2)):
        pattern.append(r"2009-\d{2}")
        pattern.append(r"2010-\d{2}")
        pattern.append(r"2011-\d{2}")
        pattern.append(r"2012-\d{2}")
        pattern.append(r"2013-\d{2}")
        pattern.append(r"2014-\d{2}")
        pattern.append(r"2015-\d{2}")
        for i in range(7):
            if re.search(pattern[i], l2[j]):
                l3.append(l2[j])
                break
        
    return l3

In [ ]:
# Get the taxi zones data, process the data to show the longitude and latitude for a given location ID
taxi_zones_df = gpd.read_file("taxi_zones/taxi_zones.shp")
taxi_zones_df = taxi_zones_df.to_crs(CRS)
taxi_zones_df['longitude'] = taxi_zones_df.centroid.x  
taxi_zones_df['latitude'] = taxi_zones_df.centroid.y

In [ ]:
# Given the url of the 'Yellow Taxi Trip Records', download the data, 
# Generate a sampling of Yellow Taxi data that’s roughly equal to the sample size of the Uber dataset
# Select useful columns and rename them
# Calculate duration of trip (trip end time - trip start time)
# If the dataset only have location ID, select rows with valid location ID
# change the location ID into longitude and latitude
# Remove the data that is not within the specified limits
# Change the measurement of duration to seconds, delete rows that have super long duration (outlier)
# Delete rows with negative duration
# Delete rows that have super long distance (outlier)
# Delete rows with negative distance
# Return the cleaned dataframe

def process_dataframe(url, taxi_zones_df):
    df = pd.read_parquet(url, engine='pyarrow')
    df = df.sample(frac =0.0002)
    if "tpep_pickup_datetime" not in df.columns:
        if "Trip_Pickup_DateTime" in df.columns:
            df = df[["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime", "Start_Lon", "Start_Lat", "End_Lon", "End_Lat", "Tip_Amt"]]
            df.rename(columns = {'Start_Lon':'pickup_longitude', 'Start_Lat':'pickup_latitude',
                                  'End_Lon':'dropoff_longitude', 'End_Lat':'dropoff_latitude'}, inplace = True)

        elif "pickup_datetime" in df.columns:
            df = df[["pickup_datetime", "dropoff_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "tip_amount"]]
            df.rename(columns = {'pickup_datetime':'Trip_Pickup_DateTime', 'dropoff_datetime':'Trip_Dropoff_DateTime',
                                  'tip_amount':'Tip_Amt'}, inplace = True)  
        
        df['Trip_Pickup_DateTime'] = df['Trip_Pickup_DateTime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        df['Trip_Dropoff_DateTime'] = df['Trip_Dropoff_DateTime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
        df['duration'] =  df['Trip_Dropoff_DateTime'] - df['Trip_Pickup_DateTime']
        
        
        
    else:
        df = df[["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID", "tip_amount"]]
        df = df[(df["PULocationID"] < 263) & (df["PULocationID"] > 0) 
               & (df["DOLocationID"] < 263) & (df["DOLocationID"] > 0)]
        df = df.merge(taxi_zones_df[['LocationID','longitude','latitude']].set_index('LocationID'),
                                    left_on='PULocationID', right_on='LocationID')
        df = df.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'})
        df = df.merge(taxi_zones_df[['LocationID','longitude','latitude']].set_index('LocationID'),
                                    left_on='DOLocationID', right_on='LocationID')
        df = df.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude'})
        df = df.drop(columns=["PULocationID", "DOLocationID"])
        df = df.rename(columns={'tpep_pickup_datetime': 'Trip_Pickup_DateTime', 'tpep_dropoff_datetime': 'Trip_Dropoff_DateTime', 'tip_amount': 'Tip_Amt'})
        df['duration'] =  df['Trip_Dropoff_DateTime'] - df['Trip_Pickup_DateTime']
        

    westlimit=-74.242330; southlimit=40.560445; eastlimit=-73.717047; northlimit=40.908524
    df = df[(df['pickup_longitude']<eastlimit) & (df['pickup_longitude']>westlimit)
        & (df['pickup_latitude']<northlimit) & (df['pickup_latitude']>southlimit)
        & (df['dropoff_longitude']<eastlimit) & (df['dropoff_longitude']>westlimit)
        & (df['dropoff_latitude']<northlimit) & (df['dropoff_latitude']>southlimit)]
    df['duration'] = df['duration'].apply(lambda x:x/np.timedelta64(1, 's'))
    df = df[(df['duration'] <= 10000) & (df['duration'] > 0)]
    df['distance'] = df.apply(lambda row: calculate_distance_with_coords(row['pickup_latitude'], row['pickup_longitude'], row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
    df = df[(df['distance'] <= 100) & (df['distance'] > 0)]
    
    return df

In [ ]:
# Getting the full dataset for taxi data
# Get dataframes from every month by function process_dataframe(url, taxi_zones_df) (defined above)
# concat them together by time order to make the full dataset

def get_full_data():
    urls = find_taxi_parquet_links()
    df = process_dataframe(urls[72], taxi_zones_df)
    for i in range(73, 84):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])
    for i in range(60, 72):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])       
    for i in range(48, 60):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])
    for i in range(36, 48):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(24, 36):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(12, 24):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])    
    for i in range(0, 6):
        df1 = process_dataframe(urls[i], taxi_zones_df)
        df = pd.concat([df, df1])     
    return df

In [ ]:
taxi_data = get_full_data()

In [ ]:
taxi_data.head()

### Processing Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    data_1 = pd.read_csv(csv_file)
    data_2 = data_1.drop(columns=['Unnamed: 0','key','fare_amount','passenger_count'])
    data_3 = data_2.dropna()
    
    # Define the range of acceptable latitude and longitude values
    min_lat, max_lat = 40.560445, 40.908524
    min_long, max_long = -74.242330, -73.717047
    
    # Remove trips that start and/or end outside of the latitude/longitude coordinate box
    data_4 = data_3.drop(
    index=data_3[
    (data_3['pickup_latitude'] < min_lat) & 
    (data_3['pickup_latitude'] > max_lat) &
    (data_3['pickup_longitude'] < min_long) &
    (data_3['pickup_longitude'] > max_long) &
    (data_3['dropoff_latitude'] < min_lat) &
    (data_3['dropoff_latitude'] > max_lat) &
    (data_3['dropoff_longitude'] < min_long) &
    (data_3['dropoff_longitude'] > max_long)
    ].index
)
    
    #normalizing and using appropriate column types for the respective data
    data_4['pickup_datetime'] = pd.to_datetime(data_3['pickup_datetime'])
    data_4 = data_4.rename(columns = {'pickup_datetime': 'Trip_Pickup_DateTime'})
    
    return data_4

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [ ]:
UBER_DATA = "uber_rides_sample.csv"

In [ ]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

### Processing Weather Data

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    weather_data=pd.read_csv(csv_file)
    weather_data['REPORT_TYPE'] = weather_data['REPORT_TYPE'].astype(str)
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
    
    # Filter out the rows where 'REPORT_TYPE' column contains the string 'SOD'
    hourly_data = weather_data.loc[~weather_data['REPORT_TYPE'].str.contains('SOD')]

    # Select only the required columns and rename them
    hourly_data = hourly_data[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    hourly_data = hourly_data.rename(columns={'DATE': 'datetime', 'HourlyPrecipitation': 'hourly_precipitation', 'HourlyWindSpeed': 'hourly_wind_speed'})

    # Convert 'HourlyPrecipitation' and 'HourlyWindSpeed' columns to numeric datatype
    hourly_data[['hourly_precipitation', 'hourly_wind_speed']] = hourly_data[['hourly_precipitation', 'hourly_wind_speed']].apply(pd.to_numeric, errors='coerce')

    # Fill NaN values in 'HourlyPrecipitation' and 'HourlyWindSpeed' columns with 0
    hourly_data[['hourly_precipitation', 'hourly_wind_speed']] = hourly_data[['hourly_precipitation', 'hourly_wind_speed']].fillna(0)
    
    return hourly_data

In [ ]:
def clean_month_weather_data_daily(csv_file):  
    weather_data = pd.read_csv(csv_file)
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE']).dt.date
    
     # Filter out the rows where 'REPORT_TYPE' column contains the string 'SOD'
    hourly_data = weather_data.loc[~weather_data['REPORT_TYPE'].str.contains('SOD')]

    # Select only the required columns
    hourly_data = hourly_data[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    
    # Convert the 'HourlyPrecipitation' and 'HourlyWindSpeed' columns to numeric format
    hourly_data['HourlyPrecipitation'] = pd.to_numeric(hourly_data['HourlyPrecipitation'], errors='coerce').fillna(0)
    hourly_data['HourlyWindSpeed'] = pd.to_numeric(hourly_data['HourlyWindSpeed'], errors='coerce').fillna(0)
    
    # Fill NaN values in 'HourlyPrecipitation' and 'HourlyWindSpeed' columns with 0
    hourly_data['HourlyPrecipitation']=hourly_data['HourlyPrecipitation'].fillna(0)
    hourly_data['HourlyWindSpeed']=hourly_data['HourlyWindSpeed'].fillna(0)
    
    # Aggregate the hourly data by date
    hourly_data = hourly_data.groupby('DATE', as_index=False).agg({'HourlyWindSpeed': 'mean', 
                                                                   'HourlyPrecipitation': 'sum'})
    
    # Filter the daily data and select the 'DATE', 'Sunrise', and 'Sunset' columns
    daily_data = weather_data[weather_data['REPORT_TYPE'] == 'SOD  ']
    daily_data=daily_data[['DATE', 'Sunrise', 'Sunset']]
    
    # Convert the 'Sunrise' and 'Sunset' columns to numeric format
    daily_data['Sunrise'] = pd.to_numeric(daily_data['Sunrise'], errors='coerce')
    daily_data['Sunset'] = pd.to_numeric(daily_data['Sunset'], errors='coerce')
    
    # Merge the hourly and daily data on the 'DATE' column
    daily_data = hourly_data.merge(daily_data, on='DATE', how='left')
    
    # Fill the data
    daily_data.fillna(method='ffill',inplace=True)
    daily_data.fillna(method='bfill',inplace=True)
    
    # Rename the columns.
    daily_data = daily_data.rename(columns={'DATE': 'date', 
                                            'HourlyPrecipitation': 'daily_precipitation', 
                                            'HourlyWindSpeed': 'daily_wind_speed', 
                                            'Sunrise': 'sunrise', 
                                            'Sunset': 'sunset'})
    
    return daily_data

In [ ]:
hourly2009=pd.DataFrame(clean_month_weather_data_hourly("weather data/2009_weather.csv"))

In [ ]:
hourly2010=pd.DataFrame(clean_month_weather_data_hourly("weather data/2010_weather.csv"))

In [ ]:
hourly2011=pd.DataFrame(clean_month_weather_data_hourly("weather data/2011_weather.csv"))

In [ ]:
hourly2012=pd.DataFrame(clean_month_weather_data_hourly("weather data/2012_weather.csv"))

In [ ]:
hourly2013=pd.DataFrame(clean_month_weather_data_hourly("weather data/2013_weather.csv"))

In [ ]:
hourly2014=pd.DataFrame(clean_month_weather_data_hourly("weather data/2014_weather.csv"))

In [ ]:
hourly2015=pd.DataFrame(clean_month_weather_data_hourly("weather data/2015_weather.csv"))

In [ ]:
hourly_data=pd.concat([hourly2009,hourly2010,hourly2011,hourly2012,hourly2013,hourly2014,hourly2015])
hourly_data.head()

In [ ]:
daily2009=pd.DataFrame(clean_month_weather_data_daily("weather data/2009_weather.csv"))

In [ ]:
daily2010=pd.DataFrame(clean_month_weather_data_daily("weather data/2010_weather.csv"))

In [ ]:
daily2011=pd.DataFrame(clean_month_weather_data_daily("weather data/2011_weather.csv"))

In [ ]:
daily2012=pd.DataFrame(clean_month_weather_data_daily("weather data/2012_weather.csv"))

In [ ]:
daily2013=pd.DataFrame(clean_month_weather_data_daily("weather data/2013_weather.csv"))

In [ ]:
daily2014=pd.DataFrame(clean_month_weather_data_daily("weather data/2014_weather.csv"))

In [ ]:
daily2015=pd.DataFrame(clean_month_weather_data_daily("weather data/2015_weather.csv"))

In [ ]:
daily_data=pd.concat([daily2009,daily2010,daily2011,daily2012,daily2013,daily2014,daily2015])
daily_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    datetime DATETIME,
    hourly_precipitation FLOAT,
    hourly_wind_speed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    date DATE,
    daily_wind_speed FLOAT,
    daily_precipitation FLOAT
    sunrise FLOAT
    sunset FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi
(
    id INTEGER PRIMARY KEY,
    Trip_Pickup_DateTime DATETIME,
    Trip_Dropoff_DateTime DATETIME,
    Total_Amt FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    duration FLOAT
)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    Trip_Pickup_DateTime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    distance FLOAT
);
"""

SUN_DATA_SCHEMA = """
CREATE TABLE IF NOT EXISTS sun_data 
(
    id INTEGER PRIMARY KEY,
    date DATE,
    sunrise INT32,
    sunset INT32
)
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(SUN_DATA_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(TAXI_TRIPS_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(UBER_TRIPS_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(DAILY_WEATHER_SCHEMA)

In [ ]:
with engine.connect() as connection:
    connection.execute(SUN_DATA_SCHEMA)

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for name, table in table_to_df_dict.items():
        table.to_sql(name, engine, if_exists='replace', index=False)

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
    "sun_data": sun_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query 1

In [ ]:
QUERY_1 = """
SELECT strftime('%H', Trip_Pickup_DateTime), count(*)
FROM taxi_trips
GROUP BY strftime('%H', Trip_Pickup_DateTime)
ORDER BY count(*) DESC;
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "Popularity of taxi rides for each hour")

### Query 2

In [ ]:
QUERY_2 = """
SELECT strftime('%w', Trip_Pickup_DateTime) AS day, COUNT(*) AS number
FROM uber_trips
GROUP BY day
ORDER BY number DESC;
"""

In [ ]:
engine.execute(QUERY_2).fetchall()

In [ ]:
write_query_to_file(QUERY_2, "Popularity of uber rides for each day")

### Query 3

In [ ]:
QUERY_3 = """
    WITH hired_trips AS
    (
    SELECT 
        Trip_Pickup_DateTime, 
        distance
    FROM
        taxi_trips
    WHERE 
        strftime('%m', Trip_Pickup_DateTime) = "07"
    AND 
        strftime('%Y', Trip_Pickup_DateTime) = "2013"
UNION
    SELECT
        Trip_Pickup_DateTime, 
        distance
    FROM
        uber_trips
    WHERE 
        strftime('%m', Trip_Pickup_DateTime) = "07"
    AND 
        strftime('%Y', Trip_Pickup_DateTime) = "2013"
    )
    SELECT distance FROM hired_trips ORDER BY distance
    LIMIT 1 OFFSET ((select count(*) from hired_trips) * 95/100 -1)    
"""

In [ ]:
engine.execute(QUERY_3).fetchall()

In [ ]:
write_query_to_file(QUERY_3, "95 percentile distance by hired trips in July 2013")

### Query 4

In [ ]:
QUERY_4 = """
    WITH hired_trips AS
    (
    SELECT 
        Trip_Pickup_DateTime, 
        distance
    FROM
        taxi_trips
    WHERE 
        strftime('%Y', Trip_Pickup_DateTime) = "2009"
UNION
    SELECT
        Trip_Pickup_DateTime, 
        distance
    FROM
        uber_trips
    WHERE 
        strftime('%Y', Trip_Pickup_DateTime) = "2009"
    )
    SELECT 
        date(Trip_Pickup_DateTime), count(*), AVG(distance)
    FROM hired_trips
    GROUP BY date(Trip_Pickup_DateTime)
    ORDER BY count(*) DESC
    LIMIT 10

"""

In [ ]:
engine.execute(QUERY_4).fetchall()

In [ ]:
write_query_to_file(QUERY_4, "Top 10 days with highest number of hired rides in 2009")

### Query 5

In [ ]:
QUERY_5 = """
    WITH hired_trips AS
    (
    SELECT DATE(Trip_Pickup_DateTime) AS date 
    FROM taxi_trips
    WHERE date BETWEEN '2014-01-01' AND '2015-01-01'
UNION
    SELECT DATE(Trip_Pickup_DateTime) AS date
    FROM uber_trips
    WHERE date BETWEEN '2014-01-01' AND '2015-01-01'
)
    SELECT 
        hired_trips.date as date, printf("%.4f",daily_wind_speed), COUNT(*) AS number
    FROM hired_trips
    JOIN 
        daily_weather ON hired_trips.date = DATE(daily_weather.date)
    GROUP BY hired_trips.date
    ORDER BY daily_wind_speed DESC
    LIMIT 10

"""

In [ ]:
engine.execute(QUERY_5).fetchall()

In [ ]:
write_query_to_file(QUERY_5, "Top 10 Windiest Date with number of hired trips in 2014")

### Query 6

In [ ]:
QUERY_6 = """
    WITH hired_trips AS
    (
    SELECT 
        strftime('%Y-%m-%d %H',Trip_Pickup_DateTime) AS trip_hour 
    FROM 
        taxi_trips
    WHERE 
        Trip_Pickup_DateTime BETWEEN '2012-10-22' AND '2012-10-31'
UNION ALL
    SELECT 
        strftime('%Y-%m-%d %H',Trip_Pickup_DateTime) AS trip_hour 
    FROM 
        uber_trips
    WHERE 
        Trip_Pickup_DateTime BETWEEN '2012-10-22' AND '2012-10-31'
        ),
    hurricane_weather AS
    (
    SELECT 
        strftime('%Y-%m-%d %H',datetime) AS weather_hour, hourly_precipitation, hourly_wind_speed 
    FROM hourly_weather
    WHERE 
        datetime BETWEEN '2012-10-22' AND '2012-10-31'
    )
    SELECT 
        weather_hour, COALESCE(COUNT(trip_hour),0) AS number, hourly_precipitation, hourly_wind_speed
    FROM hurricane_weather
    LEFT JOIN hired_trips
    ON trip_hour = weather_hour
    GROUP BY weather_hour
    
"""

In [ ]:
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_6, "Harrican Weather Data with number of hired trips")

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
def get_data_for_visual_1():
    df1 = pd.read_sql_query(QUERY_1, engine) 
    df1 = df1.sort_values(by=["strftime('%H', Trip_Pickup_DateTime)"])
    return df1

In [ ]:
def plot_popularity_hourly(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    plt.bar(df1["strftime('%H', Trip_Pickup_DateTime)"], df1["count(*)"])
    
    axes.set_title("Popularity of taxi rides for each hour")
    plt.xlabel("Hour")
    plt.ylabel("Number of Taxi Rides")
    
    plt.show()

In [ ]:
df1 = get_data_for_visual_1()
plot_popularity_hourly(df1)

### Visualization 3

In [ ]:
def get_data_for_visual_3():
    QUERY_LGA = """
    WITH hired_trips AS
    (
    SELECT 
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        taxi_trips
    WHERE dropoff_longitude < -73.854838 AND dropoff_longitude > -73.891745
        AND dropoff_latitude < 40.778865 AND dropoff_latitude > 40.763589

UNION
    SELECT
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        uber_trips
    WHERE dropoff_longitude < -73.854838 AND dropoff_longitude > -73.891745
        AND dropoff_latitude < 40.778865 AND dropoff_latitude > 40.763589
    )
    SELECT 
         day_of_week, count(*) AS LGA
    FROM hired_trips
    GROUP BY day_of_week
    ORDER BY day_of_week

"""
    QUERY_JFK = """
    WITH hired_trips AS
    (
    SELECT 
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        taxi_trips
    WHERE dropoff_longitude < -73.766264 AND dropoff_longitude > -73.795642
        AND dropoff_latitude < 40.651376 AND dropoff_latitude > 40.639263

UNION
    SELECT
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        uber_trips
    WHERE dropoff_longitude < -73.766264 AND dropoff_longitude > -73.795642
        AND dropoff_latitude < 40.651376 AND dropoff_latitude > 40.639263
    )
    SELECT 
         day_of_week, count(*) AS JFK
    FROM hired_trips
    GROUP BY day_of_week
    ORDER BY day_of_week

"""
    QUERY_EWR = """
    WITH hired_trips AS
    (
    SELECT 
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        taxi_trips
    WHERE dropoff_longitude < -74.165205 AND dropoff_longitude > -74.194028
        AND dropoff_latitude < 40.699680 AND dropoff_latitude > 40.686794

UNION
    SELECT
        strftime('%w', Trip_Pickup_DateTime) + 1 AS day_of_week, 
        dropoff_longitude,
        dropoff_latitude
    FROM
        uber_trips
    WHERE dropoff_longitude < -74.165205 AND dropoff_longitude > -74.194028
        AND dropoff_latitude < 40.699680 AND dropoff_latitude > 40.686794
    )
    SELECT 
         day_of_week, count(*) AS EWR
    FROM hired_trips
    GROUP BY day_of_week
    ORDER BY day_of_week

"""
    df1 = pd.read_sql_query(QUERY_LGA, engine) 
    df2 = pd.read_sql_query(QUERY_JFK, engine) 
    df3 = pd.read_sql_query(QUERY_EWR, engine) 
    df4 = pd.merge(df1, df2, how = "left", on = ["day_of_week", "day_of_week"])
    df5 = pd.merge(df4, df3, how = "left", on = ["day_of_week", "day_of_week"])
    return df5

In [ ]:
def plot_popularity_hourly(dataframe):
    dataframe.plot(x="day_of_week", y=["LGA", "JFK", "EWR"], kind="bar", 
                   title = "Dropoffs for Each Airport in Each Week Day",
                  xlabel = "Week Day", ylabel = "Number of Dropoffs")    

In [ ]:
df3 = get_data_for_visual_3()
plot_popularity_hourly(df3)

### Visualization 5

In [ ]:
def get_data_for_visual_5():
    QUERY = """
    SELECT Tip_Amt, distance FROM taxi_trips
    """
    df1 = pd.read_sql_query(QUERY, engine) 
    df1 = df1.sample(2000)
    # only show 2000 data so we can see the trend more clearly
    df1 = df1[df1["Tip_Amt"] < 50]
    # delete outliers with tip amount greater than 50
    df1 = df1[df1["distance"] < 50]
    # delete outliers with distance greater than 50
    return df1

In [ ]:
def Tip_Amt_VS_Distance(dataframe):
    dataframe.plot(x="distance", y="Tip_Amt", kind="scatter", 
                   title = "Tip Amount Versus Distance for Yellow Taxi Rides",
                  xlabel = "Distance", ylabel = "Tip Amount")   

In [ ]:
df5 = get_data_for_visual_5()
Tip_Amt_VS_Distance(df5)

Visualization 6

In [1]:
QUERY_tip ="""
    SELECT strftime('%Y-%m-%d %H', Trip_Pickup_DateTime) AS DATE, Tip_Amt
    FROM taxi_trips
          """ 
QUERY_precipitation ="""
    SELECT strftime('%Y-%m-%d %H', datetime) as DATE, hourly_precipitation
    FROM hourly_weather
          """

In [3]:
def tip_precipitation():
    # Load tip and precipitation data into Pandas DataFrames from a database using an engine object
    df_tip = pd.read_sql_query(QUERY_tip, engine)
    df_precipitation =pd.read_sql_query(QUERY_precipitation, engine)
    
    # Merge tip and precipitation data on the 'DATE' column and sample 2000 rows
    df = df_tip.merge(df_precipitation,on = 'DATE').sample(2000)
    
    # Filter rows where hourly precipitation and tip amount are greater than 0
    df = df[(df['hourly_precipitation'] > 0) & (df['Tip_Amt'] > 0)]
    
    # Plot a scatter plot of tip amount vs. hourly precipitation
    df.plot(x="Tip_Amt", y="hourly_precipitation", kind="scatter", 
            title="TipAmt VS. Precipitation for Yellow Taxi rides", 
            xlabel="Tip Amount", ylabel="Hourly Precipitation")

In [4]:
tip_precipitation()

NameError: name 'pd' is not defined